In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2

In [2]:
df='/kaggle/input/garbage-classification/garbage_classification'

In [6]:
new_df=[]
for cur,diry,filename in os.walk(df):
    for file in filename:
        data=os.path.join(cur,file)
        cv=cv2.imread(data)
        cv3=cv2.resize(cv,(250,250))
        if cv3 is not None:
            new_df.append(cv3)


In [3]:
import tensorflow as tf

data_dir = '/kaggle/input/garbage-classification/garbage_classification'  # Path to your data directory
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


2024-08-04 15:50:27.845241: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 15:50:27.845340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 15:50:27.985832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 15515 files belonging to 12 classes.
Using 12412 files for training.
Found 15515 files belonging to 12 classes.
Using 3103 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)  # Output: ['class_1', 'class_2']

for image_batch, labels_batch in train_ds.take(1):
    print(image_batch.shape)  # Shape of the image batch
    print(labels_batch.numpy())  # Target values (numeric labels)


In [10]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:
new_df2=np.array(new_df)

In [8]:
new_df2.shape

(15515, 250, 250, 3)

In [14]:
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.models import Model

In [16]:
inputs=Input(shape=(180,180,3))
c1=Conv2D(32,(3,3),activation="relu",padding="same")(inputs)
c2=Conv2D(32,(3,3),activation="relu",padding="same")(c1)
m1=MaxPooling2D(2,2)(c2)
dr=Dropout(0.5)(m1)
c1=Conv2D(32,(3,3),activation="relu",padding="same")(dr)
c2=Conv2D(32,(3,3),activation="relu",padding="same")(c1)
m1=MaxPooling2D(2,2)(c2)
f1=Flatten()(m1)
d1=Dense(32)(f1)
outputs=Dense(12,activation="softmax")(d1)
model=Model(inputs,outputs)

In [17]:
model.compile(optimizer='adam',loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [18]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50# You can change the number of epochs
)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 32s 67ms/step - accuracy: 0.2885 - loss: 177.9226 - val_accuracy: 0.3600 - val_loss: 1.8504
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.3940 - loss: 1.8270 - val_accuracy: 0.4173 - val_loss: 1.7410
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.4333 - loss: 1.6915 - val_accuracy: 0.4405 - val_loss: 1.8002
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.4667 - loss: 1.6127 - val_accuracy: 0.4515 - val_loss: 1.8126
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.4965 - loss: 1.5238 - val_accuracy: 0.4560 - val_loss: 1.7126
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.5216 - loss: 1.4501 - val_accuracy: 0.4811 - val_loss: 1.6441
Epoch 7/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.5448 - loss: 1.3758 - val_accuracy: 0.4931 - val_loss: 1.5959
Epoch 8/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.5575 - loss: 1.3333 

In [19]:
loss,accuracy=model.evaluate(val_ds)

97/97 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5652 - loss: 2.5548
